# Traitement des données winequality avec du ML

Importer les données

In [1]:
import numpy as np
import pandas as pd

In [2]:
white = pd.read_csv("./data/winequality-white.csv",sep=";")
red = pd.read_csv("./data/winequality-red.csv",sep=";")

In [3]:
white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
# on ajoute une colonnne avec la couleur
white["couleur"]="blanc"
red["couleur"]="rouge"

On concatène les deux jeux de données

In [5]:
# méthode 1
wine = white.append(red)

In [6]:
# méthode 2
wine = pd.concat([white, red],axis = 0)

In [7]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  couleur               6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 710.6+ KB


## On sépare les données en apprentissage / test

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# on enlève une colonne
x = wine.drop("couleur",axis=1)
y = wine["couleur"]

In [10]:
# on sépare apprentissage / test
x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y)

# Modèles de ML

In [11]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [12]:
# on crée un objet à partir de la classe du modèle
modele_rf = RandomForestClassifier(n_estimators=1000)

In [13]:
# on ajuste les paramètres du modèle en utilisant les données
modele_rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
pd.DataFrame(modele_rf.feature_importances_,index=x.columns,columns=["importance"]).sort_values("importance",ascending=False)

,importance
total sulfur dioxide,0.301394
chlorides,0.270676
volatile acidity,0.118895
sulphates,0.061085
density,0.060711
free sulfur dioxide,0.051787
fixed acidity,0.042731
residual sugar,0.042096
pH,0.020077
citric acid,0.019076


In [15]:
# on prédit sur les données de test
y_pred_rf = modele_rf.predict(x_test)

In [16]:
# mesure de la qualité du modèle
from sklearn.metrics import accuracy_score, confusion_matrix

In [17]:
accuracy_score(y_test, y_pred_rf)

0.9926153846153846

In [18]:
confusion_matrix(y_test, y_pred_rf)

array([[1224,    1],
       [  11,  389]], dtype=int64)

On cherche à automatiser le ML :

In [19]:
# on crée un dictionnaire de modèles
dico_modele = dict(rf=RandomForestClassifier(),
                   gbm = GradientBoostingClassifier(),
                   knn = KNeighborsClassifier(),
                   logit = LogisticRegression(),
                   svm =SVC())

In [20]:
def ajustement_modeles(dico_modeles, x_train, x_test, y_train, y_test):
    """ Cette fonction permet d'afficher le % de bien classés et la matrice de confusion
    pour tous les modèles du dictionnaire de modèles
    
    Input : - dico_modeles : dictionnaire des modèles de ML
            - x_train/y_train : données d'apprentissage
            - x_test/y_test : données de test
    """
    
    for nom_modele in dico_modeles.keys():
        modele = dico_modeles[nom_modele]
        modele.fit(x_train, y_train)
        y_pred = modele.predict(x_test)
        print("Pourcentage de bien classés pour {} :".format(nom_modele),accuracy_score(y_pred,y_test))
        print("Matrice de confusion pour {} :".format(nom_modele),confusion_matrix(y_pred,y_test),sep="\n")
        

In [21]:
ajustement_modeles(dico_modele, x_train, x_test, y_train, y_test)

Pourcentage de bien classés pour rf : 0.9926153846153846
Matrice de confusion pour rf :
[[1224   11]
 [   1  389]]
Pourcentage de bien classés pour gbm : 0.9907692307692307
Matrice de confusion pour gbm :
[[1224   14]
 [   1  386]]
Pourcentage de bien classés pour knn : 0.9323076923076923
Matrice de confusion pour knn :
[[1191   76]
 [  34  324]]


C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pourcentage de bien classés pour logit : 0.976
Matrice de confusion pour logit :
[[1213   27]
 [  12  373]]
Pourcentage de bien classés pour svm : 0.932923076923077
Matrice de confusion pour svm :
[[1206   90]
 [  19  310]]


In [22]:
# on prédit avec le modèle
modele_rf.predict(np.random.random(12).reshape(1, -1))

array(['blanc'], dtype=object)

In [23]:
# on peut prédire la proba avec le modèle
modele_rf.predict_proba(np.random.random(12).reshape(1, -1))

array([[0.661, 0.339]])